# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902021


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  10%|█         | 3/30 [00:06<01:01,  2.26s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<00:42,  1.65s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:30,  1.22s/it]

Rendering models:  20%|██        | 6/30 [00:07<00:21,  1.10it/s]

...

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.70it/s]

Rendering models:  93%|█████████▎| 28/30 [00:11<00:00,  5.03it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  4.79it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  4.55it/s]

equidad1                                0.086660
not-logged-in-a924460f0f1f97422ce6      0.000469
dayanakula                              0.000771
klmasters                               0.000450
kayleebug2017                           0.002055
bronzesword                             0.000609
mooberrysauce                           0.017157
tingard                                 0.000358
not-logged-in-9a813f8c400d5fedde5d      0.029340
emmahughes147                           0.007807
coyotes54                               0.000804
not-logged-in-e931dfb2080048a803b3      0.000735
jmartinez468                            0.003701
AleksanderTG                            0.002009
pangeli5                                0.001110
Saharisunshine                          0.000943
not-logged-in-214814e52004b42491a4    298.626393
haydenmanninen                          0.000446
aisha3540                               0.001698
Lavadude                                0.008205
hithereeveryone     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())